# Named Entity Recognition

## Requirements
pip install keras-tqdm
pip install git+https://www.github.com/keras-team/keras-contrib.git

# Dataset AnEM:
### Description:
"Anatomical entities such as kidney, muscle and blood are central to much of biomedical scientific discourse, and the detection of mentions of anatomical entities is thus necessary for the automatic analysis of the structure of domain texts." "The corpus consists of 500 documents (over 90,000 words) selected randomly from citation abstracts and full-text papers with the aim of making the corpus representative of the entire available biomedical scientific literature. The corpus annotation covers mentions of both healthy and pathological anatomical entities and contains over 3,000 annotated mentions." http://www.nactem.ac.uk/anatomy/ Paper: http://www.nactem.ac.uk/anatomy/docs/ohta2012opendomain.pdf

### File Visualization:


In [1]:
import pandas as pd
from IPython.display import display_html

#Try to change this variable value
sentence_to_visualize = 500

with open("./AnEM/AnEM.train", 'rb') as file_handle:
    file_content = file_handle.read().decode('utf-8').strip()
    annotated_sentences = file_content.split('\n\n')
    sentence = annotated_sentences[sentence_to_visualize]
    sentence = sentence.split()
    sentence = [sentence[i:i + 4] for i in range(0, len(sentence), 4)]
    cols=['Tokens','V1','V2','Entity']
    df2 = pd.DataFrame(sentence, columns=cols)
    df2_styler = df2.reset_index(drop=True).style.set_table_attributes("style='display:inline'").set_caption('Entities Distribution')
    display_html(df2_styler._repr_html_(), raw=True)

,Tokens,V1,V2,Entity
0,Preincubating,817,830,O
1,mononuclear,831,842,B-Cell
2,cells,843,848,I-Cell
3,with,849,853,O
4,anti,854,858,O
5,-,858,859,O
6,mycobacteria,859,871,O
7,antibodies,872,882,O
8,(,883,884,O
9,lepromatous,884,895,O


### Visual Example:
![ola](Images/sentence.png)

### Dataset Statistics:

In [ ]:
from IPython.display import display_html
import pandas as pd
import numpy as np

d = np.array([['Train', 71697, 2815], ['Test', 45939, 1882], ['All', 117636, 4697]])
cols=['Split', 'Tokens', 'Sentences']
df = pd.DataFrame(d, columns=cols)

d2 = np.array([['Anatomical_system', 51],['Cell', 776],['Cellular_component', 199],['Developing_anatomical_structure ', 39],
              ['Immaterial_anatomical_entity', 60],['Multi-tissue_structure', 639],['Organ', 381],
              ['Organism_subdivision', 162],['Organism_substance', 291],['Pathological_formation', 368],
              ['Tissue', 169],['No Entity', 112000]])
cols=['Token Entities:', 'Count']
df2 = pd.DataFrame(d2, columns=cols)

df1_styler = df.reset_index(drop=True).style.set_table_attributes("style='display:inline'").set_caption('Instances Distribution')
df2_styler = df2.reset_index(drop=True).style.set_table_attributes("style='display:inline'").set_caption('Entities Distribution')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

        

# Task:1 Prepare the dataset for the model
### Read data from a ConLL file:

In [ ]:
import os
def read_conll(filename_end):
    word_pos = 0
    pos_pos= None
    iob_pos = 3
    sep = '\t'
    IOB= 'IOB2'
    corpus_root="./AnEM"

    for root, dirs, files in os.walk(corpus_root):
        for filename in files:
            if filename.endswith(filename_end):
                with open(os.path.join(root, filename), 'rb') as file_handle:
                    try:
                        file_content = file_handle.read().decode('utf-8').strip()
                    except:
                        raise ValueError("Can't process!")
                    # Split sentences:
                    annotated_sentences = file_content.split('\n\n')

                    for annotated_sentence in annotated_sentences:
                        if annotated_sentence not in ['-DOCSTART- -X- O O', '-DOCSTART- -X- -X- O']:
                            # Split words:
                            annotated_tokens = [seq for seq in annotated_sentence.split('\n')]
                            standard_form_tokens = []

                            for idx, annotated_token in enumerate(annotated_tokens):
                                if sep=='multispace':
                                    annotations = annotated_token.split()   # Split annotations
                                else:
                                    annotations = annotated_token.split(sep)   # Split annotations
                                try:
                                    word, ner = annotations[word_pos], annotations[iob_pos]
                                except:
                                    print(annotations)
                                    #raise ValueError("??")
                                if IOB == 'IOB2':
                                    # This is for the Seminars_and_Job_postings
                                    # data:
                                    if ner=='0':
                                        ner = 'O'
                                    standard_form_tokens.append((word, ner))
                                    conll_tokens = standard_form_tokens
                                else:
                                    raise ValueError('Variable IOB has wrong value.')

                            yield [(w, iob) for w, iob in conll_tokens]
                            
                            
data_train = list(read_conll('.train'))
data_test = list(read_conll('.test'))

#We can visualize the input for each sentence:
print(data_train[0])

### Merge sentence and label vectors:

In [ ]:
def transform(data):
    sentences_array=[]
    labels_array=[]
    for data_input in data:
        sentence=[]
        labels=[]
        for vec in data_input:
            sentence.append(vec[0])
            labels.append(vec[1])
        sentences_array.append(sentence)
        labels_array.append(labels)

    return sentences_array,labels_array

sentences_train, labels_train = transform(data_train)
sentences_test, labels_test = transform(data_test)

#We can visualize the input for each sentence:
print(sentences_train[0])
print(labels_train[0])

### Convert Labels to Numeric Values:

In [5]:
def convert(labels_array):
    for idx,label_vec in enumerate(labels_array):
        for idx,label in enumerate(label_vec):
            if label=="B-Anatomical_system":
                label_vec[idx]=0
            if label=="I-Anatomical_system":
                label_vec[idx]=1
            if label=="B-Cell":
                label_vec[idx]=2
            if label=="I-Cell":
                label_vec[idx]=3
            if label=="B-Cellular_component":
                label_vec[idx]=4
            if label=="I-Cellular_component":
                label_vec[idx]=5
            if label=="B-Developing_anatomical_structure":
                label_vec[idx]=6
            if label=="I-Developing_anatomical_structure":
                label_vec[idx]=7
            if label=="B-Immaterial_anatomical_entity":
                label_vec[idx]=8
            if label=="I-Immaterial_anatomical_entity":
                label_vec[idx]=9
            if label=="B-Multi-tissue_structure":
                label_vec[idx]=10
            if label=="I-Multi-tissue_structure":
                label_vec[idx]=11
            if label=="B-Organ":
                label_vec[idx]=12
            if label=="I-Organ":
                label_vec[idx]=13
            if label=="B-Organism_subdivision":
                label_vec[idx]=14
            if label=="I-Organism_subdivision":
                label_vec[idx]=15
            if label=="B-Organism_substance":
                label_vec[idx]=16
            if label=="I-Organism_substance":
                label_vec[idx]=17
            if label=="B-Pathological_formation":
                label_vec[idx]=18
            if label=="I-Pathological_formation":
                label_vec[idx]=19
            if label=="B-Tissue":
                label_vec[idx]=20
            if label=="I-Tissue":
                label_vec[idx]=21
            if label=="O":
                label_vec[idx]=22
    return labels_array


labels_train = convert(labels_train)
labels_test = convert(labels_test)
print(labels_train[0])

[10, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22]


### Optional to convert to one-hot encoding :

In [6]:
import keras
def to_categorical(labels_array):
    for idx,label_vec in enumerate(labels_array):
        label_vec = keras.utils.to_categorical(label_vec, num_classes=23, dtype='float32')
        labels_array[idx]=label_vec
    return labels_array

labels_train2 = to_categorical(labels_train)
print(labels_train2[0])

Using TensorFlow backend.


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### Visualize Reports length:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.ticker import FuncFormatter
tlen = [len(x) for x in sentences_train] 
fig, ax = plt.subplots()
plt.hist(tlen, bins=np.arange(max(tlen)), histtype='barstacked', linewidth=2)
plt.title("Length of reports")
plt.ylabel('# of Instances', fontsize=12)
plt.xlabel('Length of reports', fontsize=12)
plt.show()

### Padding to Input Shape:

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np


tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences_train)
tokenizer.fit_on_texts(sentences_test)
voc_size = len(tokenizer.word_index)   

def convert2(x,y):
    X_total = tokenizer.texts_to_sequences(x)
    X_total = pad_sequences(X_total, maxlen=50, padding='post')
    Y_total = pad_sequences(y, maxlen=50, padding='post', value=22)
    return X_total,Y_total

X_train, y_train = convert2(sentences_train, labels_train)
X_test, y_test = convert2(sentences_test, labels_test)

print(X_train.shape)
print(X_train[0])


print(y_train.shape)
print(y_train[0])
y_train = y_train.reshape(2815,-1,1)
print(y_train.shape)
print(y_train[0])
y_test = y_test.reshape(1882,-1,1)

(2815, 50)
[1364 3121  180   10  431 1939  377 2231  603   17 3122    1    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
(2815, 50, 23)
[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  1]
 ...
 [22 22 22 ... 22 22 22]
 [22 22 22 ... 22 22 22]
 [22 22 22 ... 22 22 22]]
(2815, 1150, 1)
[[ 0]
 [ 0]
 [ 0]
 ...
 [22]
 [22]
 [22]]


### Metric Evaluation:

In [ ]:
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
def new_sparse_categorical_accuracy(y_true, y_pred):
        y_pred_rank = ops.convert_to_tensor(y_pred).get_shape().ndims
        y_true_rank = ops.convert_to_tensor(y_true).get_shape().ndims
        # If the shape of y_true is (num_samples, 1), squeeze to (num_samples,)
        if (y_true_rank is not None) and (y_pred_rank is not None) and (len(K.int_shape(y_true)) == len(K.int_shape(y_pred))):
            y_true = array_ops.squeeze(y_true, [-1])
        y_pred = math_ops.argmax(y_pred, axis=-1)
        # If the predicted output and actual output types don't match, force cast them
        # to match.
        if K.dtype(y_pred) != K.dtype(y_true):
            y_pred = math_ops.cast(y_pred, K.dtype(y_true))
        return math_ops.cast(math_ops.equal(y_true, y_pred), K.floatx())

# Attempt 1:
### Create Model:

In [ ]:
from keras.models import Model
from keras.layers import Dense,TimeDistributed, Input, Embedding,Bidirectional,LSTM,Dropout
from sklearn.utils import class_weight
import numpy as np

def create_model(voc):
    sequence_input = Input(shape=(50,), dtype='int32')
    embedded_sequences = Embedding(voc, 32, input_length=50)(sequence_input)
    bilstm = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=False))(embedded_sequences)
    dense = Dense(256)(bilstm)
    drop = Dropout(0.2)(dense)
    preds = TimeDistributed(Dense(23, activation='softmax'))(drop)
    model = Model(inputs=sequence_input, outputs=preds)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[new_sparse_categorical_accuracy])
    model.summary()
    return model

model = create_model(voc_size)

### Train:

In [ ]:
from keras_tqdm import TQDMNotebookCallback
from sklearn_crfsuite import metrics

model.fit(x=X_train, y=y_train,validation_split=0.2,batch_size=32, epochs=1,verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
y_pred = y_pred.reshape(1882,-1,1)
print(y_pred[0])
print(y_test[0])
#y_pred = y_pred.argmax(axis=-1)
print(np.unique(y_test))
print(metrics.flat_classification_report(y_test, y_pred, digits=3))



# Attempt 2:
### Create Model:

In [9]:
from keras.models import Model
from keras.layers import Dense,TimeDistributed, Input, Embedding,Bidirectional,LSTM,Dropout
from keras_contrib.layers import CRF
from sklearn.utils import class_weight
import numpy as np


def create_model(voc,weights):
    sequence_input = Input(shape=(50,), dtype='int32')
    embedded_sequences = Embedding(voc,64, input_length=50)(sequence_input)
    bilstm = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=False))(embedded_sequences)
    dense = Dense(256)(bilstm)
    drop = Dropout(0.2)(dense)
    preds = TimeDistributed(Dense(64, activation='relu'))(drop)
    crf = CRF(23)
    out = crf(preds)
    model = Model(inputs=sequence_input, outputs=out)
    model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
    model.summary()
    return model



model = create_model(voc_size,class_weight)

C:\Users\ist181159\AppData\Local\Programs\Python\Python36\lib\site-packages\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
C:\Users\ist181159\AppData\Local\Programs\Python\Python36\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 64)            755904    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 256)           197632    
_________________________________________________________________
dense_1 (Dense)              (None, 50, 256)           65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 64)            16448     
_________________________________________________________________
crf_1 (CRF)                  (None, 50, 23)            2070      
Total para

### Train:

In [11]:
from keras_tqdm import TQDMNotebookCallback
from sklearn_crfsuite import metrics

labels_train2 = to_categorical(labels_train)
print(labels_train2)
y_train = pad_sequences(labels_train2, maxlen=50, padding='post', value=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
print(y_train.shape)
print(y_train[0])
y_train = y_train.reshape(2815,-1,1)
print(y_train.shape)
print(y_train[0])
model.fit(x=X_train, y=y_train,validation_split=0.2,batch_size=128, epochs=1,verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
print(y_pred)
print(y_test)
print(metrics.flat_classification_report(y_test, y_pred, digits=3))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




(2815, 50, 23, 23, 23)
[[[[0 1 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   ...
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[0 1 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   ...
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[0 1 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   ...
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  ...

  [[0 1 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   ...
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[0 1 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   ...
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[0 1 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   ...
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]]


 [[[0 1 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   ...
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[0 1 0 ... 0 0 0]
 

HBox(children=(IntProgress(value=0, description='Training', max=1, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=2252, style=ProgressStyle(description_width='initial')), HTML(value='')))

InvalidArgumentError: Matrix size-incompatible: In[0]: [77868672,1], In[1]: [23,23]
	 [[Node: loss/crf_1_loss/MatMul_1 = MatMul[T=DT_FLOAT, _class=["loc:@training/RMSprop/gradients/loss/crf_1_loss/MatMul_1_grad/MatMul_1"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/crf_1_loss/Reshape_3, crf_1/chain_kernel/read)]]
	 [[Node: metrics/crf_viterbi_accuracy/while_1/Switch_2/_191 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2362_metrics/crf_viterbi_accuracy/while_1/Switch_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](^_cloopmetrics/crf_viterbi_accuracy/while_1/ExpandDims/dim/_73)]]

In [ ]:

from keras.models import Model
from keras.layers import Dense,TimeDistributed, Input, Embedding,Bidirectional,LSTM,Dropout
from sklearn.utils import class_weight
import numpy as np

"""
A weighted version of categorical_crossentropy for keras (2.0.6). This lets you apply a weight to unbalanced classes.
@url: https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d
@author: wassname
"""
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss
    

def create_model(voc,weights):
    sequence_input = Input(shape=(50,), dtype='int32')
    embedded_sequences = Embedding(voc, 64, input_length=50)(sequence_input)
    bilstm = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state= False))(embedded_sequences)
    dense = Dense(256)(bilstm)
    drop = Dropout(0.2)(dense)
    preds = TimeDistributed(Dense(23, activation='softmax'))(drop)
    model = Model(inputs=sequence_input, outputs=preds)
    # Compile model
    model.compile(loss=weighted_categorical_crossentropy(weights), optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

train = y_train.argmax(axis=-1)
unique, counts = np.unique(train, return_counts=True)
print(unique)
labels_dict = dict(zip(unique, counts))
total = sum(labels_dict.values())
labels_dict.update((x, total/y) for x, y in labels_dict.items())
class_weight = np.array(list(labels_dict.values()))
print(class_weight)
                        
model=create_model(voc_size,class_weight)
model.fit(x=X_train, y=y_train,validation_split=0.2,batch_size=64, epochs=10,verbose=1)


In [ ]:
from keras.models import Model
from keras.layers import Dense,TimeDistributed, Input, Embedding,Bidirectional,LSTM,Dropout
from sklearn.utils import class_weight
import numpy as np


def naive_over_sampling(x_train,y_train):
    x_train_final = x_train[:]
    y_train_final = y_train[:]
    for idx, sentence in enumerate(y_train):
        for output in sentence:
            if(np.argmax(output)!=22):
                x_train_final = np.append(x_train_final,x_train[idx]])
                y_train_final = np.append(y_train_final,sentence])
                break
    return x_train_final,y_train_final

def create_model(voc):
    sequence_input = Input(shape=(50,), dtype='int32')
    embedded_sequences = Embedding(voc, 64, input_length=50)(sequence_input)
    bilstm = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state= False))(embedded_sequences)
    dense = Dense(256)(bilstm)
    drop = Dropout(0.2)(dense)
    preds = TimeDistributed(Dense(23, activation='softmax'))(drop)
    model = Model(inputs=sequence_input, outputs=preds)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model


X_train,y_train = naive_over_sampling(X_train,y_train)
print(metrics.flat_classification_report(y_train, y_train, digits=3))
#model=create_model(voc_size)
#model.fit(x=X_train, y=y_train,validation_split=0.2,batch_size=64, epochs=10,verbose=1)

In [ ]:
from sklearn_crfsuite import metrics
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
test = y_test.argmax(axis=-1)
#sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0])) # group B and I results
print(metrics.flat_classification_report(test, y_pred, digits=3))
